# Batch convert images to dzi

## 1. Using the TissUUmaps API

Install the TissUUmaps pip package if not already installed (use `%pip install tissuumaps[full]` if you need the TissUUmaps gui):

In [ ]:
%pip install tissuumaps

Import TissUUmaps package and other dependencies:

In [1]:
import tissuumaps.views as tfv
import glob, os



INFO:root: * TissUUmaps version: 3.0.9.5



In [3]:
# Put all the images you want to convert to dzi in one folder
imageDir = "/path/to/my/tif/images/"

# Browse through the tif images and convert to dzi
for tiffImage in glob.glob(os.path.join(imageDir, "*.tif")):
    # Remove the tif extension to get the output filename:
    dziPath, _ = os.path.splitext(tiffImage)
    converter = tfv.ImageConverter(tiffImage, dziPath)
    converter.convertToDZI()

INFO:pyvips.voperation:dzsave argument dirname is deprecated
INFO:pyvips.voperation:dzsave argument dirname is deprecated
INFO:pyvips.voperation:dzsave argument dirname is deprecated
INFO:pyvips.voperation:dzsave argument dirname is deprecated


## 2. Using pyvips directly:

Install and import all dependencies:

In [2]:
%pip install pyvips

import os, glob, pyvips

Create the converter function:

In [10]:
def convertToDZI(inputImage, outputImage, jpegQuality=100):
    # create pyvips image from input:
    imgVips = pyvips.Image.new_from_file(inputImage)
    
    # rescale to 0-255 to fit in 8bit:
    minVal = imgVips.percent(0.5)
    maxVal = imgVips.percent(99.5)
    if minVal == maxVal:
        minVal = 0
        maxVal = 255
    if minVal < 0 or maxVal > 255:
        imgVips = (255.0 * (imgVips - minVal)) / (maxVal - minVal)
        imgVips = (imgVips < 0).ifthenelse(0, imgVips)
        imgVips = (imgVips > 255).ifthenelse(255, imgVips)
        imgVips = imgVips.scaleimage()
    
    # Convert to dzi:
    imgVips.dzsave(
        outputImage,
        suffix=".jpg[Q={q}]".format(
            q=jpegQuality
        ),
        background=0,
        depth="onepixel",
        overlap=0,
        tile_size=256,
    )

In [11]:
# Put all the images you want to convert to dzi in one folder
imageDir = "/path/to/my/tif/images/"

# Browse through the tif images and convert to dzi
for tiffImage in glob.glob(os.path.join(imageDir, "*.tif")):
    # Remove the tif extension to get the output filename:
    dziPath, _ = os.path.splitext(tiffImage)
    converter = convertToDZI(tiffImage, dziPath, jpegQuality=100)